<a href="https://colab.research.google.com/github/Spark095/11101ML/blob/main/ML_0616.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from urllib.request import urlretrieve
url = "https://raw.githubusercontent.com/Elwing-Chou/tibaml0606/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://raw.githubusercontent.com/Elwing-Chou/tibaml0606/main/titanic/test.csv"
urlretrieve(url, "test.csv")

# from urllib.request import urlretrieve
# url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/train.csv"
# urlretrieve(url, "train.csv")
# url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/test.csv"
# urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f13991e3d90>)

In [94]:
# import pandas as pd
# #train_df = pd.read_csv("train.csv", encoding="utf-8")
# train_df = pd.read_csv("train.csv", encoding="utf-8")
# predict_df = pd.read_csv("test.csv", encoding="utf-8")


import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
predict_df = pd.read_csv("test.csv", encoding="utf-8")

In [95]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [96]:
datas = pd.concat([train_df, predict_df], axis=0, ignore_index=True)
datas = datas.drop(['PassengerId',"Survived"], axis=1)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [97]:
# 資料預處理
# Step1. 填補空值
s = datas.isna().sum()
#print(s)
# 找序列中每一個不等於0
s != 0
# 把這個True False 放進去篩選
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [98]:
# 1.2 check 每一個欄位是那一種型態
# a. 數值 Age, SibSp, Parch, Ticket(#share this ticket), Fare, 
# b. 類別： Pclass, Name(mid), Sex, Cabin(first), Embarked.
# 補空值（最可能出現的值）
# 類別： 最常出現的值 
# 數值: 絕對不是平均值， 而是中位數!!! 不是因為平均對中位數有什麼影響！ 他們本來就是兩種不同統計值， 我們的需求是最可能的值（不大不小的值）

In [99]:
# 處理一下 ticket/name/cabin
def cabinchange(c):
  if pd.isna(c):
     return None
  else:
     return c[0]

datas["Cabin"] = datas["Cabin"].apply(cabinchange)


In [100]:
counts = datas["Ticket"].value_counts()
def ticketchange(t):
    if pd.isna(t):
        return None
    else:
        return counts[t]
datas["Ticket"] = datas["Ticket"].apply(ticketchange)

In [101]:
#n = "Braund, Mr. Owen Harris"
def namechange(n):
    return n.split(",")[-1].split(".")[0].strip()

datas["Name"] = datas["Name"].apply(namechange)

In [102]:
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,None,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C,C
2,3,Miss,female,26.0,0,0,1,7.9250,None,S
3,1,Mrs,female,35.0,1,0,2,53.1000,C,S
4,3,Mr,male,35.0,0,0,1,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,Mr,male,NaN,0,0,1,8.0500,None,S
1305,1,Dona,female,39.0,0,0,3,108.9000,C,C
1306,3,Mr,male,38.5,0,0,1,7.2500,None,S
1307,3,Mr,male,NaN,0,0,1,8.0500,None,S


In [103]:
# 類別 補最常見的 most show 出最常見的ID
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)



In [104]:
# check 還缺多少
s = datas.isna().sum()
s[s != 0].sort_values(ascending=False)


Cabin    1014
Age       263
Fare        1
dtype: int64

In [105]:
# 補數值 中位數
# Pclass 不是數值，不能補中位數，要drop
med = datas.median().drop(["Pclass"])
# 會follow上面的字典，自動把數值空值補相對應的中位數
datas = datas.fillna(med)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [106]:
# one-hot 一定要做，但是為了美觀， 我可以把太過稀少的去除
count = datas["Name"].value_counts()
reserved = count[count >50].index
def namefilter(mid):
    if mid in reserved:
      return mid
    else:
      return None
datas["Name"] = datas["Name"].apply(namefilter)

In [111]:
# Pclass 很特別： 雖然他是類別，但是他的類別是有大小關係的， 所以可以不做 one-hot encoding
# 做不做各有各的好處，但是這個案例，做比較好
# Sex可以不做: 因為是二值型
pd.set_option('display.max_columns',30)
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [108]:
import re
n = "Braund, Mr. Owen Harris"
pat = ".+,(.+)\..+"
re.match(pat, n).group(1).strip()

'Mr'

In [109]:
# pandas篩選: 篩選[針對每一筆給出True/False]
df = pd.DataFrame([
    [1, 2],
    [3, 4]
])
df[[False, True]]
s = pd.Series(["A", "B", "C"])
s[[True, True, False]]

0    A
1    B
dtype: object

In [110]:
# pandas sum
df = pd.DataFrame([
    [1, 2],
    [3, 4]
])
df.sum(axis=1)

0    3
1    7
dtype: int64